# A1: Logic and lambda calculus

Simon Dobnik and Robin Cooper

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on how to work in groups on Canvas.

Write all your answers and the code in the appropriate boxes below.

*Important*: We recommend that you create a virtual environment (either with virtualenv or conda), install Jupyter Lab in it and all dependencies used in the assignment.
To run Jupyter Lab later within your environment, run the following command:

```python -m ipykernel install --user --name=my-virtualenv-name```,

where you replace `my-virtualenv-name` with the name of your created environment.
Once in Jupyter, choose the kernel with the name of your environment. You can do it by either (i) using the drop-down menu in the top right corner or (ii) going to the top menu -> Kernel -> Change Kernel.

## Translating English to logic and evaluating logic in a model

In [4]:
# This task needs NLTK and Jupyter Lab (IPython package).
import nltk
from utils import display_latex, display_translation, display_tree, display, Markdown
read_expr = nltk.sem.Expression.fromstring

/home/devix/miniconda3/envs/Python_School/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 1. Propositional logic
Translate the following sentences into **propositional logic** and verify that they parse with Expression.fromstring() (`read_expr` variable in the cell above). Provide a key which shows how the propositional variables in your translation correspond to expressions of English. Briefly discuss any difficulties you encounter. (By difficulties we mean cases where the semantics of English expressions cannot be expressed to the same degree by the semantics of your logic representations, i.e. they do not mean the same). **[5 + 1 marks]**

In [5]:
propositions = {
    "Alex plays the piano":
    read_expr('p'),

    "Alex is smart":
    read_expr('q'),

    "If Alex plays the piano, she is smart.":
    read_expr('p -> q'),
    
    "Alex is musical":
    read_expr('m'),

    "Alex is both smart and musical.":
    read_expr('q & m'),
    
    "Lydia is not happy":
    read_expr('-h'),
    
    "If Alex is not smart, Lydia is not happy.":
    read_expr('-q -> -h'),
    
    "George plays the piano":
    read_expr("g"),

    "George is musical":
    read_expr("l"),

    "If Alex or George plays the piano, they are musical.":
    read_expr('(p -> m) | (g -> l)'),
    
    "George plays the piano.":
    read_expr('g'),
}

for text, semrep in propositions.items():
    display_translation(text, semrep)

"Alex plays the piano": $p$

"Alex is smart": $q$

"If Alex plays the piano, she is smart.": $(p\ \rightarrow\ q)$

"Alex is musical": $m$

"Alex is both smart and musical.": $(q\ \land\ m)$

...

"If Alex is not smart, Lydia is not happy.": $(-q\ \rightarrow\ -h)$

"George plays the piano": $g$

"George is musical": $l$

"If Alex or George plays the piano, they are musical.": $((p\ \rightarrow\ m)\ \lor\ (g\ \rightarrow\ l))$

"George plays the piano.": $g$

*Difficulties encountered:*

### 2. Valuation of Propositional logic

Imagine that we observe a world where 
- (i) Alex does not play the piano,
- (ii) Alex and Lydia are smart and musical,
- (iii) George is not musical,
- (iv) Lydia is happy,
- (v) George plays the piano. 

Translate this informal description of the world into a model by appropriately defining an evaluation function and evaluate the formulae from Question 1 in this model. Briefly comment the answers you get. **[5 + 1 marks]**.

In [6]:
# answers here

print("Logic model - atomized propositions based on input sentences:")
propositions = {
"Alex plays the piano":
    read_expr('p'),
'Lydia is smart':
    read_expr('f'),
'Lydia is musical':
    read_expr('v'),
"Alex is smart":
read_expr('q'),
"Alex is musical":
read_expr('m'), 
'George is musical':
    read_expr('l'),
'Lydia is happy':
    read_expr('h'),
'George plays the piano':
    read_expr('g')
}


for text, semrep in propositions.items():
    display_translation(text, semrep)
print("Logic model - evaluation function based on input sentences:")
propositions = {
"p": read_expr('False'),
"f": read_expr('True'),
"v": read_expr("True"),
"q": read_expr("True"),
"m": read_expr("True"),
"l": read_expr("False"),
"h": read_expr("True"),
"g": read_expr("True")
}



for text, semrep in propositions.items():
    display_translation(text, semrep)

print("Evaluating the previous expressions:")

val = nltk.Valuation([('p', False), ('f', True), ('v', True), ('q', True), 
                       ('m', True), ('l', False), ('h', True), ('g', True),])
dom = set([]) 
g = nltk.Assignment(dom) 
m = nltk.Model(dom, val)


propositions = {
"If Alex plays the piano, she is smart.":
    read_expr(str(m.evaluate('p -> q', g))),

"Alex is both smart and musical.":
read_expr(str(m.evaluate('q & m', g))),

"If Alex is not smart, Lydia is not happy.":
read_expr(str(m.evaluate('-q -> -h', g))),

"If Alex or George plays the piano, they are musical.":
    read_expr(str(m.evaluate('(p -> m) | (g -> l)', g))),

"George plays the piano.":
read_expr(str(m.evaluate('g', g))),
}

for text, semrep in propositions.items():
    display_translation(text, semrep)




Logic model - atomized propositions based on input sentences:


"Alex plays the piano": $p$

"Lydia is smart": $f$

"Lydia is musical": $v$

"Alex is smart": $q$

...

"If Alex plays the piano, she is smart.": $True$

"Alex is both smart and musical.": $True$

"If Alex is not smart, Lydia is not happy.": $True$

"If Alex or George plays the piano, they are musical.": $True$

"George plays the piano.": $True$

*Comments:*

### 3. Predicate logic *without quantifiers*

Translate the following sentences into predicate-argument formulae of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [7]:
sentences1 = {
    "Lydia likes George but Lydia doesn't like Alex": 
    read_expr(r'like(lydia, george) & -like(lydia, alex)'),
    
    "Lydia likes herself and so does George":
    read_expr(r'like(lydia, lydia) & like(george, george)'),
    
    "Charlie is an English pianist who plays a sonata":
    read_expr(r'english(charlie) & pianist(charlie) & play(charlie, sonata)'),

    
    "Lydia and George admire each other":
    read_expr(r'admire(lydia, george) & admire(george, lydia)'),
}

for text, semrep in sentences1.items():
    display_translation(text, semrep)


"Lydia likes George but Lydia doesn't like Alex": $(like(lydia,george)\ \land\ -like(lydia,alex))$

"Lydia likes herself and so does George": $(like(lydia,lydia)\ \land\ like(george,george))$

"Charlie is an English pianist who plays a sonata": $(english(charlie)\ \land\ pianist(charlie)\ \land\ play(charlie,sonata))$

"Lydia and George admire each other": $(admire(lydia,george)\ \land\ admire(george,lydia))$

*Difficulties encountered:*

### 4. First order logic with quantifiers

Translate the following sentences into quantified formulas of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [8]:
sentences2 = {
    "Charlie knows a woman who likes George":
    read_expr('exists x.(woman(x) & like(x, george) & know(charlie, x))'),
    
    "George admires everybody and Lydia admires nobody":
    read_expr('all x.(person(x) -> (admire(george, x) & -admire(lydia, x)))'),

    "Nobody admires everybody":
    read_expr('-exists x.(person(x) & all y.(person(y) -> admire(x, y)))'),
    
    "Exactly one musician plays everything Alex wrote":
    read_expr('exists x.(musician(x) & all y.(wrote(alex, y) -> play(x, y)) & all z.(musician(z) & all y.(wrote(alex, y) -> play(z, y)) -> x=z))'),
}

for text, semrep in sentences2.items():
    display_translation(text, semrep)

"Charlie knows a woman who likes George": $\exists\ x.(woman(x)\ \land\ like(x,george)\ \land\ know(charlie,x))$

"George admires everybody and Lydia admires nobody": $\forall\ x.(person(x)\ \rightarrow\ (admire(george,x)\ \land\ -admire(lydia,x)))$

"Nobody admires everybody": $-\exists\ x.(person(x)\ \land\ \forall\ y.(person(y)\ \rightarrow\ admire(x,y)))$

"Exactly one musician plays everything Alex wrote": $\exists\ x.(musician(x)\ \land\ \forall\ y.(wrote(alex,y)\ \rightarrow\ play(x,y))\ \land\ \forall\ z.((musician(z)\ \land\ \forall\ y.(wrote(alex,y)\ \rightarrow\ play(z,y)))\ \rightarrow\ (x\ =\ z)))$

*Difficulties encountered:*

### 5. Valuation of first order logic

We observe a world with entities Lydia, George, Alex, Charlie and Bertie, sonata, etude, prelude, waltz, scherzo.

1. Lydia likes Lydia, George, Alex and Charlie. George likes Lydia, Bertie and George. Alex likes Alex. Charlie likes Lydia, George, Alex, Charlie and Bertie. Bertie likes Alex.
2. Lydia, George, Alex, Charlie and Bertie are English.
3. Charlie and Bertie are pianists.
4. Charlie plays a sonata, an etude and a waltz. Bertie plays a waltz and a scherzo. Lydia plays an etude, a prelude and a waltz.
5. Lydia admires Lydia, Charlie and Bertie. George admires Lydia, George, Alex, Charlie and Bertie. Alex admires Lydia, Alex and Bertie. Charlie admires George and Bertie. Bertie admires Lydia, George, Alex, Charlie and Bertie.
6. Lydia knows Lydia, George, Alex, Charlie and Bertie. George knows Lydia, George and Bertie. Alex knows Lydia, Alex and Bertie. Charlie knows George, Charlie and Bertie. Bertie knows Lydia, George, Alex, Charlie and Bertie.
7. Lydia, Alex and Charlie are women.
8. George and Bertie are men.
9. Alex wrote a sonata, an etude an a waltz.
10. Lydia, Alex, Charlie and Bertie are musicians.

Translate this informal description of the world into a model and evaluate the formulae from Questions 3 and 4 in this model. Briefly comment on the answers you get **[3 + 2 marks]**.

In [9]:
entities = set(['l','g','a','c','b','s','e','p','w','z'])

assign = """
    lydia => l
    george => g
    alex => a
    charlie => c
    bertie => b
    sonata => s
    etude => e
    prelude => p
    waltz => w
    scherzo => z
    english => {l, g, a, c, b}
    person => {l, g, a, c, b}
    musician => {l, a, c, b}
    pianist => {c, b}
    woman => {l, a, c}
    man => {g, b}
    like => {(l, l), (l, g), (l, a), (l, c), (g, l), (g, b), (g, g), (a, a), (c, l), (c, g), (c, a), (c, c), (c, b), (b, a)}
    admire => {(l, l), (l, c), (l, b), (g, l), (g, g), (g, a), (g, c), (g, b), (a, l), (a, a), (a, b), (c, g), (c, b), (b, l), (b, g),(b, a) (b, c), (b, b)}
    know => {(l, l), (l, g), (l, a), (l, c), (l, b), (g, l), (g, g), (g, b), (a, l), (a, a), (a, b), (c, g), (c, c), (c, b), (b, l), (b, g), (b, a), (b, c), (b, b)}
    play => {(c, s), (c, e), (c, w), (b, w), (b, z), (l, e), (l, p), (l, w)}
    wrote => {(a, s), (a, e), (a, w)}
"""

val2 = nltk.Valuation.fromstring(assign)

g2 = nltk.Assignment(entities)
m2 = nltk.Model(entities, val2)

# sentences from question 3
for text, semrep in sentences1.items():
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))

# sentences from question 4
for text, semrep in sentences2.items():
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))

False


$(like(lydia,george)\ \land\ -like(lydia,alex))$

----

True


$(like(lydia,lydia)\ \land\ like(george,george))$

...

$-\exists\ x.(person(x)\ \land\ \forall\ y.(person(y)\ \rightarrow\ admire(x,y)))$

----

True


$\exists\ x.(musician(x)\ \land\ \forall\ y.(wrote(alex,y)\ \rightarrow\ play(x,y))\ \land\ \forall\ z.((musician(z)\ \land\ \forall\ y.(wrote(alex,y)\ \rightarrow\ play(z,y)))\ \rightarrow\ (x\ =\ z)))$

----

*Comments on the answers:*

## Lambda calculus

In [7]:
from nltk.grammar import FeatureGrammar

### 6. Function application and $\beta$-reduction
In the following examples some code has been deleted and replaced with `<????>`. What has been deleted? Verify that your answer is correct. **[4 marks]**

In [ ]:
e1 = read_expr(r'\x.like(x,rob)')
e2 = read_expr(r'pip')
e3 = nltk.sem.ApplicationExpression(e1,e2) 
display_latex(e3.simplify())
# with result like(pip,rob).
display_latex(read_expr(r"like(pip,rob)"))

e1 = read_expr(r'\P.P(pip)')
e2 = read_expr(r'\x.play(x,scherzo)') 
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result play(pip,scherzo).
display_latex(read_expr(r"play(pip,scherzo)"))

e1 = read_expr(r'\P.exists x.(woman(x) & P(x))')
e2 = read_expr(r'\x.play(x,etude)') 
e3 = nltk.sem.ApplicationExpression(e1,e2) 
display_latex(e3.simplify())
# with result exists x.(woman(x) & play(x,etude)).
display_latex(read_expr(r"exists x.(woman(x) & play(x,etude))"))

e1 = read_expr(r'\Q.\x.Q(\y.like(x,y))')
e2 = read_expr(r'\P.all x. (musician(x) -> P(x))')
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result \x.all z2.(musician(z2) -> like(x,z2)).
display_latex(read_expr(r"\x.all z2.(musician(z2) -> like(x,z2))"))

### 7. Extending the grammar

Extend the grammar simple_sem.fcfg that comes with NLTK `(~/nltk_data/grammars/book_grammars/)` so that it will cover the following sentences:

- no man gives a bone to a dog **[4 marks]**
- no man gives a bone to the dog **[4 marks]**
- a boy and a girl chased every dog **[2 marks]**
- every dog chased a boy and a girl **[2 marks]**
- a brown cat chases a white dog **[4 marks]**

The last example includes adjectives. Several different kinds of adjectives are discussed in the literature [(cf. Kennedy, 2012)](http://semantics.uchicago.edu/kennedy/docs/routledge.pdf). In this example we have an intersective adjective. The denotiation we want for "brown cat" is a a set that we get by intersecting the set of individuals that are brown and the set of individuals that are cats.

C. Kennedy. Adjectives. In G. Russell, editor, The Routledge Companion to Philosophy of Language, chapter 3.3, pages 328–341. Routledge, 2012.

The original grammar is included in the code below as a string.

In [9]:
fcfg_string_orginal = r"""
% start S
############################
# Grammar Rules
#############################

S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]

VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]

PP[+TO, SEM=?np] -> P[+TO] NP[SEM=?np]

#############################
# Lexical Rules
#############################

PropN[-LOC,NUM=sg,SEM=<\P.P(angus)>] -> 'Angus'
PropN[-LOC,NUM=sg,SEM=<\P.P(cyril)>] -> 'Cyril'
PropN[-LOC,NUM=sg,SEM=<\P.P(irene)>] -> 'Irene'
 
Det[NUM=sg,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'every'
Det[NUM=pl,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'all'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'some'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'an'

N[NUM=sg,SEM=<\x.man(x)>] -> 'man'
N[NUM=sg,SEM=<\x.girl(x)>] -> 'girl'
N[NUM=sg,SEM=<\x.boy(x)>] -> 'boy'
N[NUM=sg,SEM=<\x.bone(x)>] -> 'bone'
N[NUM=sg,SEM=<\x.ankle(x)>] -> 'ankle'
N[NUM=sg,SEM=<\x.dog(x)>] -> 'dog'
N[NUM=pl,SEM=<\x.dog(x)>] -> 'dogs'

IV[NUM=sg,SEM=<\x.bark(x)>,TNS=pres] -> 'barks'
IV[NUM=pl,SEM=<\x.bark(x)>,TNS=pres] -> 'bark'
IV[NUM=sg,SEM=<\x.walk(x)>,TNS=pres] -> 'walks'
IV[NUM=pl,SEM=<\x.walk(x)>,TNS=pres] -> 'walk'
TV[NUM=sg,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chases'
TV[NUM=pl,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chase'
TV[NUM=sg,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'sees'
TV[NUM=pl,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'see'
TV[NUM=sg,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bites'
TV[NUM=pl,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bite'
DTV[NUM=sg,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'give'

P[+to] -> 'to'
"""

Write your extension of this grammar here:

In [10]:
fcfg_string = fcfg_string_orginal + r"""
Det[NUM=sg,SEM=<\P Q.-(exists x.(P(x) & Q(x)))>] -> 'no'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x) & all y.(P(y) -> (y = x)))>] -> 'the'
CONJ[SEM=<\P Q.\x.(P(x) | Q(x))>] -> 'and'
NP[NUM=pl,SEM=<\P.(?np1(P) & ?np2(P))>] -> NP[SEM=?np1] CONJ[SEM=?conj] NP[SEM=?np2]
N[NUM=sg,SEM=<\x.cat(x)>] -> 'cat'
TV[NUM=sg,SEM=<\X x.X(\ y.chase(x,y))>,TNS=past] -> 'chased'
TV[NUM=pl,SEM=<\X x.X(\ y.chase(x,y))>,TNS=past] -> 'chased'
ADJ[SEM=<\P x.(brown(x) & P(x))>] -> 'brown'
ADJ[SEM=<\P x.(white(x) & P(x))>] -> 'white'
Nom[NUM=?n,SEM=<?adj(?nom)>] -> ADJ[SEM=?adj] Nom[NUM=?n,SEM=?nom]
"""

# Load `fcfg_string` as a feature grammar:
syntax = FeatureGrammar.fromstring(fcfg_string)

Run the code below without errors:

In [ ]:
# comment out sentences if you couldn't find an answer for them
sentences = [
    'no man gives a bone to a dog',
    'no man gives a bone to the dog',
    'a boy and a girl chased every dog',
    'every dog chased a boy and a girl',
    'a brown cat chases a white dog',
]
for results in nltk.interpret_sents(sentences, syntax):
    for (synrep, semrep) in results:
        display(Markdown('----'))
        display_latex(semrep) # prints the SEM feature of a tree
        display_tree(synrep) # show the parse tree

If you are working with iPython which is also running behind Jupyter notebooks and you are changing grammars and want to rerun a new version without restarting you may find `nltk.data.clear_cache()` useful.

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

## Marks

The assignment is marked on a 7-level scale where 4 is sufficient to complete the assignment; 5 is good solid work; 6 is excellent work, covers most of the assignment; and 7: creative work. 

This assignment has a total of 47 marks. These translate to grades as follows: 1 = 17% 2 = 34%, 3 = 50%, 4 = 67%, 5 = 75%, 6 = 84%, 7 = 92% where %s are interpreted as lower bounds to achieve that grade.